# Week 4 Assignment

This week's assignment is to test a logistic regression model.

## Data Management

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

# bug fix for display formats to avoid run time errors
pd.set_option('display.float_format', lambda x:'%.2f'%x)

df = pd.read_csv('gapminder.csv')

# convert to numeric format
df['incomeperperson'] = pd.to_numeric(df['incomeperperson'], errors='coerce')
df['polityscore'] = pd.to_numeric(df['polityscore'], errors='coerce')
df['urbanrate'] = pd.to_numeric(df['urbanrate'], errors='coerce')

# listwise deletion of missing values
subset = df[['incomeperperson', 'polityscore', 'urbanrate']].dropna()

### Convert Polity Score

Since there are 21 polity score categories I chose to compress them into the 5 categories specified by the Polity IV project author's.

In [2]:
# This function converts the polity score to a category
def convert_polityscore_to_category(polityscore):
    if polityscore == 10:
        return 1
    else:
        return 0

# Now we can use the function to create the new variable
subset['full_democracy'] = subset['polityscore'].apply(convert_polityscore_to_category)

counts = subset.groupby('full_democracy').size()
print(counts)

full_democracy
0    123
1     32
dtype: int64


### Create High Income Binary Variable (Response Variable)

Since I am doing a logistic regression I need to bin up the income variable into two categories.

In [3]:
# Create a threshold
income_threshold = np.mean(subset['incomeperperson'])
print(income_threshold)

6604.574358


In [4]:
# Set binary flag that income per person is greater than the threshold
def income_higher_than_threshold(income):
    if income > income_threshold:
        return 1
    else:
        return 0

subset['high_income'] = subset['incomeperperson'].apply(income_higher_than_threshold)

counts = subset.groupby('high_income').size()
print(counts)

high_income
0    116
1     39
dtype: int64


### Create Urbanization Binary Variable (Possible Confounding Response Variable)

Since I am doing a logistic regression I need to bin up the urbanization variable into two categories.

In [5]:
# Create a threshold
urbanization_threshold = np.mean(subset['urbanrate'])
print(urbanization_threshold)

55.1077419355


In [6]:
# Set binary flag that urbanization rate is greater than the threshold
def urbanrate_higher_than_threshold(urbanrate):
    if urbanrate > urbanization_threshold:
        return 1
    else:
        return 0

subset['high_urbanrate'] = subset['urbanrate'].apply(urbanrate_higher_than_threshold)

counts = subset.groupby('high_urbanrate').size()
print(counts)

high_urbanrate
0    71
1    84
dtype: int64


## Logistic Regression Model

Now I will create a simple logistic regression model that will test the relationship between being in a higher than average income class and the type of society.

In [7]:
# logistic regression with society type
lreg1 = smf.logit(formula = 'high_income ~ full_democracy', data = subset).fit()
print (lreg1.summary())

Optimization terminated successfully.
         Current function value: 0.389711
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:            high_income   No. Observations:                  155
Model:                          Logit   Df Residuals:                      153
Method:                           MLE   Df Model:                            1
Date:                Sat, 19 Dec 2015   Pseudo R-squ.:                  0.3091
Time:                        12:23:56   Log-Likelihood:                -60.405
converged:                       True   LL-Null:                       -87.436
                                        LLR p-value:                 1.944e-13
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -2.0523      0.284     -7.229      0.000        -2.609    -1.496
full_democracy   

In [8]:
# odd ratios with 95% confidence intervals
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (np.exp(conf))

                Lower CI  Upper CI    OR
Intercept           0.07      0.22  0.13
full_democracy     10.17     76.04 27.81


### Is Urban Rate a Confounder?

In [9]:
# logistic regression with society type and urbanization rate
lreg2 = smf.logit(formula = 'high_income ~ full_democracy + high_urbanrate', data = subset).fit()
print (lreg2.summary())

Optimization terminated successfully.
         Current function value: 0.342689
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:            high_income   No. Observations:                  155
Model:                          Logit   Df Residuals:                      152
Method:                           MLE   Df Model:                            2
Date:                Sat, 19 Dec 2015   Pseudo R-squ.:                  0.3925
Time:                        12:23:56   Log-Likelihood:                -53.117
converged:                       True   LL-Null:                       -87.436
                                        LLR p-value:                 1.246e-15
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -3.5056      0.636     -5.509      0.000        -4.753    -2.258
full_democracy   

In [10]:
# odd ratios with 95% confidence intervals
params = lreg2.params
conf = lreg2.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print (np.exp(conf))

                Lower CI  Upper CI    OR
Intercept           0.01      0.10  0.03
full_democracy      5.88     49.67 17.09
high_urbanrate      2.47     35.16  9.31
